In [1]:
# Importa bibliotecas
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import string
import pickle
import re

import nltk
from nltk import tokenize
from nltk import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# stop-words em português
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ptbr = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Obtém letra do site https://www.letras.mus.br/ 

def obtem_letra(link):
   
  page = requests.get(link).text # Obtém HTML da página 
  soup = BeautifulSoup(page, "lxml") # Interpreta como XML

  letra = ""
  # Obtém textos contidos em parágrafos dentro do conteiner da letra completa
  for p in soup.find(class_="cnt-letra").find_all('p'):
    letra += " " + p.get_text(" ") # Concatena textos e substitui <br> por um espaço
  
  return letra

In [4]:
letra = obtem_letra('https://www.letras.mus.br/marilia-mendonca/todo-mundo-vai-sofrer/')
letra

' A garrafa precisa do copo O copo precisa da mesa A mesa precisa de mim E eu preciso da cerveja Igual eu preciso dele na minha vida Mas quanto mais eu vou atrás, mais ele pisa Então já que é assim Se por ele eu sofro sem pausa Quem quiser me amar Também vai sofrer nessa bagaça Quem eu quero, não me quer Quem me quer, não vou querer Ninguém vai sofrer sozinho Todo mundo vai sofrer Quem eu quero, não me quer Quem me quer, não vou querer Ninguém vai sofrer sozinho Todo mundo vai sofrer Igual eu preciso dele na minha vida Mas quanto mais eu vou atrás, mais ele pisa Então já que é assim Se por ele eu sofro sem pausa Quem quiser me amar Também vai sofrer nessa bagaça Quem eu quero, não me quer Quem me quer, não vou querer Ninguém vai sofrer sozinho Todo mundo vai sofrer Quem eu quero, não me quer Quem me quer, não vou querer Ninguém vai sofrer sozinho Todo mundo vai sofrer Quem eu quero, não me quer Quem me quer, não vou querer Ninguém vai sofrer sozinho Todo mundo vai sofrer Quem eu quero,

In [5]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')

# Função para pré-processar o texto
def pre_processar(texto):

  texto = texto.lower() # minúsculas
  texto = re.sub('\[.*?\]', '', texto) # remove texto entre colchetes 
  texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto) # remove pontuação
  texto = re.sub('\w*\d\w*', '', texto) # remove palavras contendo números
  texto = sent_tokenizer.tokenize(texto)

  return texto

In [11]:
letra = pre_processar(letra)
letra

[' a garrafa precisa do copo o copo precisa da mesa a mesa precisa de mim e eu preciso da cerveja igual eu preciso dele na minha vida mas quanto mais eu vou atrás mais ele pisa então já que é assim se por ele eu sofro sem pausa quem quiser me amar também vai sofrer nessa bagaça quem eu quero não me quer quem me quer não vou querer ninguém vai sofrer sozinho todo mundo vai sofrer quem eu quero não me quer quem me quer não vou querer ninguém vai sofrer sozinho todo mundo vai sofrer igual eu preciso dele na minha vida mas quanto mais eu vou atrás mais ele pisa então já que é assim se por ele eu sofro sem pausa quem quiser me amar também vai sofrer nessa bagaça quem eu quero não me quer quem me quer não vou querer ninguém vai sofrer sozinho todo mundo vai sofrer quem eu quero não me quer quem me quer não vou querer ninguém vai sofrer sozinho todo mundo vai sofrer quem eu quero não me quer quem me quer não vou querer ninguém vai sofrer sozinho todo mundo vai sofrer quem eu quero não me quer

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Cria instância
vect = TfidfVectorizer()

In [13]:
X = vect.fit_transform(letra)
X

<1x49 sparse matrix of type '<class 'numpy.float64'>'
	with 49 stored elements in Compressed Sparse Row format>

In [14]:
print(vect.get_feature_names())

['amar', 'assim', 'atrás', 'bagaça', 'cerveja', 'copo', 'da', 'de', 'dele', 'do', 'ele', 'então', 'eu', 'garrafa', 'igual', 'já', 'mais', 'mas', 'me', 'mesa', 'mim', 'minha', 'mundo', 'na', 'nessa', 'ninguém', 'não', 'pausa', 'pisa', 'por', 'precisa', 'preciso', 'quanto', 'que', 'quem', 'quer', 'querer', 'quero', 'quiser', 'se', 'sem', 'sofrer', 'sofro', 'sozinho', 'também', 'todo', 'vai', 'vida', 'vou']


In [15]:
print(X.toarray())

[[0.048795   0.048795   0.048795   0.048795   0.0243975  0.048795
  0.048795   0.0243975  0.048795   0.0243975  0.09759001 0.048795
  0.31716752 0.0243975  0.048795   0.048795   0.09759001 0.048795
  0.34156503 0.048795   0.0243975  0.048795   0.14638501 0.048795
  0.048795   0.14638501 0.29277002 0.048795   0.048795   0.048795
  0.07319251 0.07319251 0.048795   0.048795   0.34156503 0.29277002
  0.14638501 0.14638501 0.048795   0.048795   0.048795   0.34156503
  0.048795   0.14638501 0.048795   0.14638501 0.34156503 0.048795
  0.19518001]]


In [16]:
# Mostra os valores em um Dataframe do pandas
pd.set_option('display.max_rows', None)
df = pd.DataFrame(X.T.todense(), index=vect.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False).head(10)

,tfidf
vai,0.341565
quem,0.341565
me,0.341565
sofrer,0.341565
eu,0.317168
não,0.292770
quer,0.292770
vou,0.195180
ninguém,0.146385
mundo,0.146385
